# Algoritmo - Obtendo possíveis locais de embarque
### _Autor: Antônio Claudio Dutra Batista_
### _Orientador: Francisco Moraes de Oliveria Neto_
### _Metodologia: Critério de validação após o embarque e verificação da demanda em trecho especifico_

# 1.0 Importação de dados da bilhetagem
    ''' Importação e filtragem do trecho de interesse para obtenção dos possiveis locias de embarque '''

In [ ]:
# Importando dataframe com validações georreferenciadas (dia de interesse)
import pandas as pd
bilhetagem = pd.read_csv("2018-11-01_geo.csv", usecols= ['id', 'linha', 'nome_linha', 'prefixo_carro', 'nome_cartao', 'sentido_viagem', 'momento', 'id_veiculo', 'latitude', 'longitude', 'cod_veiculo'], sep=',')

In [ ]:
bilhetagem

In [ ]:
# Filtrando validações no trecho de interesse
''' (A PARTIR DA ESTAÇÃO DO NORTHSHOPING E 120 METROS APOS A ULTIMA ESTAÇÃO) '''
validacoes_trecho = bilhetagem[(bilhetagem.latitude>=-3.736013)
    & (bilhetagem.longitude>=-38.566300) & (bilhetagem.longitude<=-38.542240) 
                                & (bilhetagem.latitude<=-3.731021)]

In [ ]:
linha = validacoes_trecho[validacoes_trecho.linha==26]

In [ ]:
linha

In [ ]:
# Filtrando de horario especifico (caso necessário)
# validacoes_trecho = bilhetagem[(bilhetagem.momento>='2018-11-01 18:00:00') & (bilhetagem.momento<='2018-11-01 19:00:00')]

In [ ]:
# Criando coluna gemetrica 
from shapely import Point
validacoes_trecho['geometry'] = validacoes_trecho.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)

In [ ]:
# Criando um geodataframe com os dados de localização das paradas no trecho
import geopandas as gpd 
validacoes_trecho = gpd.GeoDataFrame(validacoes_trecho, geometry='geometry')

In [ ]:
validacoes_trecho

# 2.0 Importação dos dados do GTFS 
    ''' Importação da base com paradas e suas respectivas localizações para filtragem das paradas no trecho de interesse '''

In [ ]:
# Importando base com locais previstos de parada
stops_paradas = pd.read_csv('stops.txt', sep= ',')

In [ ]:
# Removendo colunas desnecessárias do df com paradas 
stops_paradas.drop(columns= ['stop_code', 'stop_desc', 'zone_id', 'stop_url', 'location_type', 'parent_station', 'stop_timezone', 'wheelchair_boarding'], inplace=True)

In [ ]:
# Filtrando paradas no trecho de interesse
paradas_trecho = stops_paradas[(stops_paradas.stop_lat>=-3.736070)
    & (stops_paradas.stop_lon>=-38.566688) & (stops_paradas.stop_lon<=-38.543328) 
                                & (stops_paradas.stop_lat<=-3.731264)]

In [ ]:
# Criando coluna gemetrica 
paradas_trecho['geometry'] = paradas_trecho.apply(lambda x: Point((float(x.stop_lon), float(x.stop_lat))), axis=1)

In [ ]:
# Criando um geodataframe com os dados de localização das paradas no trecho
paradas_trecho = gpd.GeoDataFrame(paradas_trecho, geometry='geometry')

In [ ]:
paradas_trecho

In [1]:
''' Visualizando paradas no trecho '''

' Visualizando paradas no trecho '

In [ ]:
# importando biblioteca folium e plugins
import folium 
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster

# Criando media para visulização do mapa proximo a região dos dados geometricos
lon = paradas_trecho['stop_lon'].mean()
lat = paradas_trecho['stop_lat'].mean()

In [ ]:
# Definindo a função para criar o conteúdo do popup
def popup_content(stop_name):
    return f"<b>Nome da Parada:</b> {stop_name}"

# Criando um mapa centrado em FORTALEZA
mapa = folium.Map(location=[lat, lon], zoom_start=5)

# Adicionando marcadores circulares ao mapa
marker_cluster = MarkerCluster().add_to(mapa)

for index, row in paradas_trecho.iterrows():
    folium.CircleMarker(location=[row['stop_lat'], row['stop_lon']],
                        radius=10,
                        color='black',
                        fill=True,
                        fill_color='red',
                        fill_opacity=0.6,
                        popup=folium.Popup(popup_content(row['stop_name']), max_width=300)).add_to(marker_cluster)

In [ ]:
mapa

In [2]:
''' Como no trecho de interesse somente tem informações da localizaçao das estações, então vamos deixar somente estas, pois são as que serão utilizadas no metodo para estimação do possivel lical de validação! '''

' Como no trecho de interesse somente tem informações da localizaçao das estações, então vamos deixar somente estas, pois são as que serão utilizadas no metodo para estimação do possivel lical de validação! '

In [ ]:
paradas_estacoes = paradas_trecho[paradas_trecho['stop_name'].str.contains('ESTAÇÃO ')]

In [ ]:
paradas_estacoes

# 3.0 Estimando possiveis locais de validação
    ''' Criterio que para cada validação pegando a estação anterior mais proxima adimitindo que esse seja o possivel local de validação '''

In [ ]:
from shapely.geometry import Point

# Função para encontrar a estação mais próxima para uma determinada validação de ônibus
def encontrar_estacao_anterior(validacoes_trecho, paradas_estacoes):
    ponto_validacao = validacoes_trecho['geometry']
    
    # Variáveis para armazenar a estação mais próxima e sua distância
    estacao_mais_proxima = None
    menor_distancia = float('inf') # Inicializando com um valor grande
    
    # Iterando sobre cada linha no DataFrame de estações
    for estacao_index, estacao in paradas_estacoes.iterrows():
        ponto_estacao = estacao['geometry']
        distancia = ponto_validacao.distance(ponto_estacao)
        
        # Verificando se as estações são mais próxima do que as anteriores
        if distancia < menor_distancia:
            menor_distancia = distancia
            estacao_mais_proxima = ponto_estacao
    
    return estacao_mais_proxima

In [ ]:
# Aplique a função a cada linha do DataFrame de validações de ônibus
validacoes_trecho['estacao_anterior'] = validacoes_trecho.apply(lambda x: encontrar_estacao_anterior(x, paradas_estacoes), axis=1)

In [ ]:
validacoes_trecho

## 4.0 Contagem de validações por estação 

In [ ]:
# Contagem da quantidade de validações por estações 
quantidade_validacao = gpd.GeoDataFrame(validacoes_trecho['estacao_anterior'].value_counts())

In [ ]:
# Contagem da quantidade de validações por estações
contagem_validacoes = gpd.GeoDataFrame(validacoes_trecho['estacao_anterior'].value_counts())
contagem_validacoes.columns = ['quantidade_validacoes']

# Para colocar as informações da estação anterior e a contagem em colunas separadas
contagem_validacoes['estacao_anterior'] = contagem_validacoes.index

# Resetando o índice
contagem_validacoes.reset_index(drop=True, inplace=True)

In [ ]:
contagem_validacoes

In [ ]:
# Realizabdo dentifcação das estações pela lat, long
validacoes_estacao = pd.merge(contagem_validacoes, paradas_estacoes, left_on='estacao_anterior',
                              right_on="geometry")

In [ ]:
# Df com contagem de validações nas estações do trecho de interesse 
validacoes_estacao

# ------------------------------------------------------